# Introduction
The purpose of this file is to compress our LiDAR .tif file (size: 1.49 GB) down to a smaller size that ChatGPT can interface with (< 20 MB).

In [1]:
# imports
from pathlib import Path
import numpy as np
import rasterio
import matplotlib as mpl
from rasterio.enums import Resampling
from matplotlib import pyplot as plt
from matplotlib.colors import LightSource

In [2]:
# Set the INFILE variable to the filepath of the LiDAR .tif file you want to compress
# Set the OUT_PNG variable to the filepath you want the new, compressed image to appear at.

INFILE  = Path("/Users/jarrett/Documents/openai_amazon/data/lidar/raw/Inkpkio_NW_LIDAR.tif")
OUT_PNG = Path("/Users/jarrett/Documents/openai_amazon/data/lidar/bronze/preview.png")

# Tweak the SCALE variable until the resulting image is 10-20 MB
SCALE   = 0.20        # keep 20 % of the width/height
CMAP    = "terrain"

# 1) Read + down-sample
with rasterio.open(INFILE) as src:
    arr = src.read(
        1,
        out_shape=(1,
                   int(src.height * SCALE),
                   int(src.width  * SCALE)),
        resampling=Resampling.bilinear)
    nodata = src.nodata

# 2) Mask no-data
arr = np.ma.masked_equal(arr, nodata) if nodata is not None else np.ma.masked_invalid(arr)

# 3) Contrast-stretch
vmin, vmax = np.percentile(arr.compressed(), (2, 98))
norm = np.clip((arr - vmin) / (vmax - vmin), 0, 1)

# 4) Hill-shade with proper colormap object
ls   = LightSource(azdeg=315, altdeg=45)
cmap = mpl.colormaps.get_cmap(CMAP)
rgb  = ls.shade(norm, cmap=cmap, vert_exag=1, blend_mode="overlay")

# 5) Save
plt.imsave(OUT_PNG, rgb)
print("Saved:", OUT_PNG, round(OUT_PNG.stat().st_size / 1e6, 2), "MB")

Saved: /Users/jarrett/Documents/openai_amazon/data/lidar/bronze/preview.png 14.66 MB
